<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>GLM Fraud Detection with Python and Teradata SQL</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'>
    In recent years we have seen a massive increase in Fraud attempts, making fraud detection necessary for Banking and Financial Institutions. Despite countless efforts and human supervision, hundreds of millions are lost due to fraud. Fraud can happen using various methods, i.e., stolen credit cards, misleading accounting, phishing emails, etc. Due to small cases in significant populations, the detection of fraud is essential as well as challenging.
    <br>
    <br>
    This notebook provides a demonstration of "data science workflow" that illustrates how to leverage <b>teradataml</b> package to build, validate and score a model at scale in Vantage without moving the data. Users can perform large-scale operations such as feature analysis, data transformation, Model training and ML Model Scoring in the Vantage environment without moving data.</p>


<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Read the data from Vantage as a teradataml Dataframe</li>
    <li>Clean up the dataset</li>
    <li>Create training and testing datasets in Vantage</li>
    <li>In-Database GLM model training</li>
    <li>In-Database GLM model scoring</li>
    <li>Visualize the results (ROC curve and AUC)</li>
    <li>Cleanup</li>
</ol>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Installing some dependencies</b>

In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages
!pip install --upgrade teradataml

<p style = 'font-size:16px;font-family:Arial'>
    <i><b>*BEFORE proceeding, please RESTART the kernel to bring new software into Jupyter.</b></i>
</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Configuring the Environment</b>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import os
import getpass
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from teradataml.dataframe.dataframe import DataFrame
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.dataframe.dataframe import in_schema
from teradataml.context.context import create_context, remove_context
from teradataml import db_drop_table
from teradataml import ROC

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

warnings.filterwarnings("ignore")
%matplotlib inline

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute("SET query_band='DEMO=GLM_Fraud_Detection_InDB.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('demo_glm_fraud_cloud');"        # Takes 10 seconds
# %run -i ../run_procedure.py "call get_data('demo_glm_fraud_local');"        # Takes 30 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used. </p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Read the data from Vantage as a teradaml Dataframe</b>
<p style = 'font-size:16px;font-family:Arial'>The data from <a href = 'https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data'>https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data</a> is loaded in Vantage in a table named "transaction_data". Check the data size and print sample rows: 63k rows and 12 columns.</p>
<p style = 'font-size:16px;font-family:Arial'>Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</p>

In [ ]:
txn_data = DataFrame(in_schema('DEMO_GLM_Fraud','transaction_data'))

print(txn_data.shape)

# head() is possible through teradataml.
txn_data.head(5)

<p style = 'font-size:16px;font-family:Arial'>Here we rename a misspelt column without moving the data out of Vantage. We are renaming <b>oldbalanceOrg</b> to <b>oldbalanceOrig</b></p>

In [ ]:
new_data = txn_data.assign(oldbalanceOrig = txn_data.oldbalanceOrg).drop(['oldbalanceOrg'] , axis=1)

# head() is possible through teradataml.
new_data.head(5)

<p style = 'font-size:16px;font-family:Arial'>Fraudulent agents inside a simulation make these transactions. In this specific dataset, the fraudulent behaviour of the agents aims to profit by taking control or customers' accounts and trying to empty the funds by transferring them to another account and then cashing out of the system.</p>
<p style = 'font-size:16px;font-family:Arial'><b>Below are some insights about the dataset:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>There are 92 fraud transactions i.e. 0.14% of fraud transactions in the dataset.</li>
    <li>From these 92 fraud transactions, 47 are of type TRANSFER and 45 are of type CASH_OUT.</li>
    <li>97.83% of fraud transations have transaction amount equal to oldbalanceOrig i.e. account cleanout.</li>
    <li>71.74% of fraud transactions have recipient's old balance as zero.</li>
    <li>isFlaggedFraud is correct only two times among the 92 fraud transactions.</li>
</ol>

In [ ]:
new_data_pandas = new_data.to_pandas()

In [ ]:
new_data_pandas.head()

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Clean up the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>Based on what we discovered above, we will:</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Remove all data that isn't 'CASH OUT' or 'TRANSFER'.</li>
    <li>Drop "nameOrig" and "nameDest" since the origin and destination accounts don't matter.</li>
    <li>Drop "isFlaggedFraud" because it has just flagged two transactions. Hence it doesn't have much significance.</li>
</ol> 

In [ ]:
clean_data = new_data.loc[(new_data.type == 'CASH_OUT') | (new_data.type == 'TRANSFER')]
clean_data.shape

<p style = 'font-size:16px;font-family:Arial'>Now our dataset is reduced to 27k records.</p>

In [ ]:
clean_data = clean_data.drop(['nameDest', 'nameOrig', 'isFlaggedFraud'], axis = 1)
clean_data.head(5)

<p style = 'font-size:16px;font-family:Arial'>Here we'll copy this Teradata Dataframe to a separate table in Vantage.</p>

In [ ]:
#create the source data table in the database
clean_data.to_sql('clean_data', if_exists = 'replace', primary_index='txn_id')

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Create training and testing datasets in Vantage</b>

<p style = 'font-size:16px;font-family:Arial'><b>We'll perform the following steps:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Feature scaling using ScaleFit and ScaleTransform</li>
    <li>Splitting the data in training and testing datasets (80:20 split)</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'>ScaleFit outputs a table of statistics used as an input to ScaleTransform, which scales specified input table columns. ScaleTransform scales specified input table columns using ScaleFit output.</p>

<p style = 'font-size:16px;font-family:Arial'>Feature scaling is performed during data pre-processing to handle highly varying magnitudes, values, or units. If feature scaling is not done, then a machine learning algorithm tends to weigh greater values higher and consider smaller values as lower ones, regardless of the unit of the values.</p>

In [ ]:
from teradataml import ScaleFit, ScaleTransform

sf_fit = ScaleFit(data = clean_data, scale_method = 'STD',
                     target_columns = ['step', 'amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig'])

sf_trns = ScaleTransform(data = clean_data, object = sf_fit.output, accumulate = ["txn_id", "isFraud"])
sf_trns.result.head(5)

In [ ]:
tdf_samples = sf_trns.result.sample(frac = [0.2, 0.8])
copy_to_sql(tdf_samples[tdf_samples['sampleid'] == 2], table_name = 'clean_data_train', schema_name = 'demo_user', if_exists = 'replace')
copy_to_sql(tdf_samples[tdf_samples['sampleid'] == 1], table_name = 'clean_data_test', schema_name = 'demo_user', if_exists = 'replace')

In [ ]:
# pd.read_sql('select * from clean_data_train sample 5;', eng)
# df = DataFrame.from_query('select * from clean_data_train sample 5;')
df = DataFrame.from_table('clean_data_train') 
df.head()

<p style = 'font-size:16px;font-family:Arial'>The above output shows that the data has transformed into a scaled dataset. Scaling of data makes it easy for the model to learn and understand the problem.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. In-Database GLM model training</b>
<p style = 'font-size:16px;font-family:Arial'>The GLM function is a generalized linear model (GLM) that performs regression and classification analysis on data sets, where the response follows an exponential or binomial family distribution.</p>
<p style = 'font-size:16px;font-family:Arial'>Due to gradient-based learning, the function is highly sensitive to feature scaling. Input features should be standardized, such as using ScaleFit, and ScaleTransform, before using them in the function. The function takes only numeric features. We must convert the categorical features to numeric values before training. The function skips the rows with missing (null) values during training.</p>

In [ ]:
from teradataml import GLM, TDGLMPredict

glm_model = GLM(data = DataFrame('"demo_user"."clean_data_train"'),
                input_columns = '2:7', 
                response_column = 'isFraud',
                family = 'Binomial')

In [ ]:
glm_model.result

<p style = 'font-size:16px;font-family:Arial'>The function output is a trained GLM model, which can be input to the TDGLMPredict function for prediction. The model also contains model statistics of MSE, Loglikelihood, AIC, and BIC.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. In-Database GLM model scoring</b>
<p style = 'font-size:16px;font-family:Arial'>The TDGLMPredict function predicts target values (regression) and class labels (classification) for test data using a GLM model trained by the GLM function.</p>
<p style = 'font-size:16px;font-family:Arial'>Similar to GLM, input features should be standardized, such as using ScaleFit, and ScaleTransform, before using them in the function. The function takes only numeric features. We must convert the categorical features to numeric values before prediction.</p>

In [ ]:
glm_prediction = TDGLMPredict(newdata = DataFrame('"demo_user"."clean_data_test"'),
                           id_column = 'txn_id',
                           object = glm_model.result,
                           accumulate = 'isFraud',
                           output_prob=True,
                           output_responses = ['0', '1'])

In [ ]:
# This can be achieved using teradataml
print(glm_prediction.result.head(5))

<p style = 'font-size:16px;font-family:Arial'>The output above shows prob_1, i.e. transaction is fraud and prob_0, i.e. transaction is not a fraud. The prediction column uses these probabilities to give a class label, i.e. prediction column.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>8. Visualize the results (ROC curve and AUC)</b>
<p style = 'font-size:16px;font-family:Arial'>Calculate mean absolute error and AUC(Area Under the Curve) for Receiver Operating Characteritic Curve</p>
<p style = 'font-size:16px;font-family:Arial'>Mean Absolute Error is the summation of the difference between actual and predicted values averaged over the number of observations.</p>

In [ ]:
pred = glm_prediction.result.to_pandas()
print(mean_absolute_error(pred['isFraud'], pred['prob_1']))

<p style = 'font-size:16px;font-family:Arial'>The ROC curve is a graph between TPR(True Positive Rate) and FPR(False Positive Rate). The area under the ROC curve is a metric of how well the model can distinguish between positive and negative classes. The higher the AUC, the better the model's performance in distinguishing between the positive and negative classes. AUC above 0.75 is generally considered decent.</p>

In [ ]:
AUC = roc_auc_score(pred['isFraud'], pred['prob_1'])
AUC

In [ ]:
roc_out = ROC(probability_column='"prob_1"',
              observation_column="isFraud",
              positive_class="1",
              data=glm_prediction.result)

In [ ]:
roc_out

In [ ]:
fpr, tpr, thresholds = roc_curve(pred['isFraud'], pred['prob_1'])
plt.plot(fpr, tpr, color='orange', label='ROC. AUC = {}'.format(str(AUC)))
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

<p style = 'font-size:16px;font-family:Arial'>Looking at the above ROC Curve, we can confidently say that the model has performed well on testing data. The AUC value is way above 0.75 and resonates with our understanding that the model is performing well.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>9. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
# db_drop_table() drops the table from the given schema. If schema is not specified then 
# function drop the table from current schema. 
db_drop_table('clean_data') 

In [ ]:
db_drop_table('clean_data_train') 

In [ ]:
db_drop_table('clean_data_test') 

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('demo_glm_fraud');"        # Takes 5 seconds

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Dataset:</b>

- `txn_id`: transaction id
- `step`: maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (31 days simulation).
- `type`: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
- `amount`: amount of the transaction in local currency
- `nameOrig`: customer who started the transaction
- `oldbalanceOrig`: customer's balance before the transaction
- `newbalanceOrig`: customer's balance after the transaction
- `nameDest`: customer who is the recipient of the transaction
- `oldbalanceDest`: recipient's balance before the transaction
- `newbalanceDest`: recipient's balance after the transaction
- `isFraud`: identifies a fraudulent transaction (1) and non fraudulent (0)
- `isFlaggedFraud`: flags illegal attempts to transfer more than 200,000 in a single transaction

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Uses a dataset and feature discovery methods outlined here: <a href = 'https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook'>https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook</a></li>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>